# 实验2

In [1]:
import time
import random
import re
import warnings
import numpy as np

from fastHan import FastHan
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D, Bidirectional

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
warnings.filterwarnings("ignore")

Using TensorFlow backend.


## 1. 数据预处理

### 1.1 加载label

In [3]:
print(0)
labels = []
labels_list = []
with open('./label.txt', 'r', encoding='UTF-8') as f:
    for line in f:
        labels.append(eval(line))#把所有标签（有重复）放到一个list 加eval就知道这是字符串“的值”
        labels_list.extend(eval(line))#每次extend一个list 加eval就知道这是字符串

0


In [3]:
labels_list = dict(enumerate(set(labels_list)))
labels_list = {value:key for key,value in labels_list.items()}
labels_len = len(labels_list)
labels_len

233

总共233个类别

In [4]:
labels[:10]

[[],
 ['产品价格', '产品销量', '公司营销'],
 ['季度报告'],
 ['股票市场', 'A股'],
 ['IPO'],
 ['行业未来前景', '季度报告'],
 ['股票市场', 'A股'],
 ['公司业务数据', '业绩快报', '财务绩效'],
 ['政府征求意见'],
 []]

In [5]:
len(labels)

60158

### 1.2 加载title

In [6]:
titles = []
with open('./title.txt', 'r', encoding='UTF-8') as f:
    for line in f:
        titles.append(line.strip().strip('"'))

In [7]:
titles[:10]

['唯品会的特卖之道',
 '苏宁超市双十一暴涨287%，吃喝不愁更不在乎价格',
 '信而富去年亏损6658万美元，称正有序退出网贷业务',
 '快讯：得利斯跌停  报于9.55元',
 '三启赴港上市路 十二年“恩怨情仇” 阿里巴巴终圆梦香港',
 '新能源车产业链Q3盈利能力：动力电池最强 资源最弱',
 '国元证券：预估甬金股份发行价22.52元 对应市盈率16.24倍',
 '国信证券：腾讯控股新业务发展快速，广告下行',
 '图解“股票定向发行规则（征求意见稿）”',
 '曾鸣：创业的每一天，都是悬崖边的狂欢！']

In [8]:
len(titles)

60158

### 1.3 加载content

In [9]:
contents = []
with open('./content.txt', 'r', encoding='UTF-8') as f:
    for line in f:
        if line != "\n":
            contents.append(line.strip('"').strip().split('######'))

In [10]:
contents[3]

['金融界网站11月15日讯 今日得利斯开盘报10.27元，截止13:05分，该股跌9.99%报9.55元，封上跌停板。 ',
 '昨日（2019-11-14）该股净流出金额5729.8万元，主力净流出5825.78万元，中单净流入22.94万元，散户净流入73.04万元。',
 '最近一个月内，得利斯共计登上龙虎榜3次，表明得利斯股性一般。 ',
 '公司主要从事 生猪屠宰']

In [11]:
len(contents)

60158

### 1.4 分词、去除停用词

In [12]:
stopwords = [line.strip() for line in open('./stopwords.txt', 'r', encoding='utf-8').readlines()]
Han = FastHan(model_type='large')
length = len(labels)
X = []
y = []
array_zero = [0 for index in range(length)]

loading vocabulary file C:\Users\yinzh\.fastNLP\fasthan\fasthan_large\vocab.txt
Load pre-trained BERT parameters from file C:\Users\yinzh\.fastNLP\fasthan\fasthan_large\model.bin.


In [13]:
for content in contents:
    for c in content:
        if len(c) > 500:
            print(c)

430422永继电气545833954飞天经纬76430430普滤得546833960华发教育77430432方林科技547833972司南导航78430437食安科技548833994翰博高新79430455德联科技549833997巴鲁特80430457三网科技550834013利和兴81430458陆海科技551834019大自然82430459华艺园林552834020东霖食品83430462树业环保553834021流金岁月84430472安泰得554834023金投金融85430485旭建新材555834070盛全服务86430488东创科技556834082中建信息87430489佳先股份557834102电联股份88430492老来寿558834122云端网络89430500亚奥科技559834126联益科技90430505上陵牧业560834134中业科技91430508中视文化561834153炜田新材92430512芯朋微562834154建为历保93430515麟龙股份563834156有米科技94430523泰谷生物564834178金田铜业95430539扬子地板565834179赛科星96430552亚成微566834187储吉信息97430555英派瑞567834195华清飞扬98430556雅达股份568834203华澜微99430578差旅天下569834206傲基电商100430595唐人通服570834209正合股份101430596新达通571834222迈动医疗102430597博安通572834240中广瑞波103430607大树智能573834255上讯信息104430609中磁视讯574834262康富科技105430618凯立德575834270远大特材106430651金豹实业576834303华龙期货107430659江苏铁发577834342慧云股份108430665高衡力578834365杭州掌盟109430675天跃科技579834385力港网络110430680联兴科技580834415恒拓开源111430682中天羊业581834425新赛点112430714奇才股份582834428蓝孚高能113430718合肥高科583834438良晋电商114430730先大药业584834440怡丽科姆115

In [4]:
for i in range(length):
    if labels[i] == []:
        continue
    # print(i)
    array_tmp = array_zero
    for label in labels[i]:
        array_tmp[labels_list[label]] = 1
    y.append(array_tmp)
    # 分词
    cons = Han(titles[i])
    cons.extend(Han(contents[i]))
    x = []
    # 去除停用词
    for con in cons:
        for c in con:
            if c not in stopwords:
                x.append(c)
    X.append(x)

NameError: name 'length' is not defined

In [25]:
labels[1]

['产品价格', '产品销量', '公司营销']